This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/trajectories.html).  I recommend having both windows open, side-by-side!

In [3]:
from functools import partial

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    InverseKinematics,
    Parser,
    Rgba,
    RigidTransform,
    Solve,
    Sphere,
    StartMeshcat,
)

from pydrake.examples import ManipulationStation

# from pydrake.manipulation import running_as_notebook
# from pydrake.manipulation.scenarios import AddPlanarIiwa, AddShape, AddWsg
# from pydrake.manipulation.utils import ConfigureParser

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# Visualizing the configuration space

I've got the default sampling resolution set fairly fine; it will take a minute or so to run.

In [8]:
def draw_configuration_space(shelves=True):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddPlanarIiwa(plant)
    wsg = AddWsg(plant, iiwa, roll=0.0, welded=True)
    sphere = AddShape(plant, Sphere(0.02), "sphere")
    X_WO = RigidTransform([0.6, 0, 0.65])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("sphere"), X_WO)

    if shelves:
        parser = Parser(plant)
        ConfigureParser(parser)
        bin = parser.AddModelsFromUrl("package://manipulation/shelves.sdf")[0]
        plant.WeldFrames(
            plant.world_frame(),
            plant.GetFrameByName("shelves_body", bin),
            RigidTransform([0.6, 0, 0.4]),
        )

    plant.Finalize()

    #    visualizer = MeshcatVisualizer.AddToBuilder(
    #        builder,
    #        scene_graph,
    #        meshcat,
    #        MeshcatVisualizerParams(delete_prefix_initialization_event=False))

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    sphere_frame = plant.GetFrameByName("sphere", sphere)

    ik = InverseKinematics(plant, plant_context)
    collision_constraint = ik.AddMinimumDistanceLowerBoundConstraint(0.001, 0.01)
    grasp_constraint = ik.AddPositionConstraint(
        gripper_frame, [0, 0.1, 0], sphere_frame, [0, 0, 0], [0, 0, 0]
    )

    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print("IK failed")

    diagram.ForcedPublish(context)

    def eval(q0, q1, q2, c, tol):
        return float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))

    meshcat.Delete()
    meshcat.SetProperty("/Background", "visible", False)
    meshcat.SetObject("initial_guess", Sphere(0.05), Rgba(0.4, 1, 1, 1))
    meshcat.SetTransform("initial_guess", RigidTransform(q0))
    meshcat.SetObject("ik_solution", Sphere(0.05), Rgba(0.4, 0.4, 1, 1))
    meshcat.SetTransform("ik_solution", RigidTransform(result.GetSolution(q)))

    low = plant.GetPositionLowerLimits()
    up = plant.GetPositionUpperLimits()
    N = 70 if running_as_notebook else 5
    vertices, triangles = mcubes.marching_cubes_func(
        tuple(low),
        tuple(up),
        N,
        N,
        N,
        partial(eval, c=grasp_constraint, tol=0.05),
        0.5,
    )
    meshcat.SetTriangleMesh(
        "grasp_constraint", vertices.T, triangles.T, Rgba(0.5, 0.9, 0.5)
    )

    if shelves:
        vertices, triangles = mcubes.marching_cubes_func(
            tuple(low),
            tuple(up),
            N,
            N,
            N,
            partial(eval, c=collision_constraint, tol=0.0),
            0.5,
        )
        meshcat.SetTriangleMesh(
            "collision_constraint",
            vertices.T,
            triangles.T,
            Rgba(0.9, 0.5, 0.5, 1),
        )


# I've made pymcubes optional (it's a heavy dependency)
try:
    import mcubes
    print("GOOOOOD")

    
except ImportError:
    pass


draw_configuration_space(shelves=True)

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/fe5326c5ffc36fda12c58883d22d29dc86009d65.tar.gz


NameError: name 'mcubes' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fcb7160b-9d0d-4ddc-a502-dc2d27c2eb0d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>